# 손동작 분류 경진대회

## 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/손동작-분류/Data/train.csv')

In [4]:
df.shape

(2335, 34)

In [7]:
df.drop('id', axis=1, inplace=True)

## 간단한 모델링

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split

In [8]:
def accuracy(true, pred):   
    score = np.mean(true == pred)
    return score

In [9]:
dt = df.copy()

In [10]:
target = dt['target']
dt.drop('target', axis = 1, inplace = True)

In [11]:
skfold = StratifiedKFold(n_splits = 5,  shuffle = True, random_state = 22)

In [50]:
cb = CatBoostClassifier(random_state = 22, learning_rate = 0.03, n_estimators = 750, verbose = 0, max_depth = 6)

In [48]:
scores = cross_validate(cb, dt, target, return_train_score = True, scoring = 'accuracy', cv = skfold)

In [14]:
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9795503211991434, 0.8261241970021412)

In [28]:
xgb = XGBClassifier(random_state = 22, learning_rate = 0.03, n_estimators = 500, max_depth = 6, eval_metric = 'logloss')

In [29]:
scores = cross_validate(xgb, dt, target, return_train_score = True, scoring = 'accuracy', cv = skfold)

In [30]:
np.mean(scores['train_score']), np.mean(scores['test_score'])

(1.0, 0.8021413276231263)

In [18]:
lgbm = LGBMClassifier(random_state = 22, learning_rate = 0.03, n_estimators = 700, max_depth = 5)

In [19]:
scores = cross_validate(lgbm, dt, target, return_train_score = True, scoring = 'accuracy', cv = skfold)

In [20]:
np.mean(scores['train_score']), np.mean(scores['test_score'])

(1.0, 0.8098501070663812)

## test 예측

`-` catboost 사용

In [31]:
test = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/손동작-분류/Data/test.csv')

In [32]:
test.shape

(9343, 33)

In [34]:
test.drop('id', axis=1, inplace=True)

In [35]:
cb.fit(dt, target)

In [36]:
pred = cb.predict(test)

In [41]:
pred.shape

(9343, 1)

In [42]:
submission = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/손동작-분류/Data/sample_submission.csv')

In [44]:
submission['target'] = pred

In [45]:
submission.head()

,id,target
0,1,0
1,2,0
2,3,1
3,4,3
4,5,2


In [52]:
submission.to_csv('C:/Users/Jaesu/github_desktop/Dacon-Basic/손동작-분류/Data/submission.csv', index=False)